In [16]:
import torch
import pandas as pd

import lightning as L
from torch.utils.data import TensorDataset, DataLoader
import model as m


model = m.LightningLSTM()

In [17]:
df =  pd.read_csv("Gasleak Data Sets/sensor_readings.csv")
df.head()

,Unnamed: 0,time,111111_ 40.595561_-105.14055_3,111111_ 40.596108_-105.140583_4,111111_40.595556_-105.140069_2,111111_40.596114_-105.140075_1,222222_ 40.596108_-105.140583_4,222222_40.595556_-105.140069_2,222222_40.595561_-105.14055_3,222222_40.596114_-105.140075_1,...,444444_40.595881_-105.139686_1,444444_40.595947_-105.139833_4,555555_40.595542_-105.139211_2,555555_40.595547_-105.139714_3,555555_40.596089_-105.139144_1,555555_40.596097_-105.139678_4,666666_40.595542_-105.139211_2,666666_40.595547_-105.139714_3,666666_40.596089_-105.139144_1,666666_40.596097_-105.139678_4
0,1,1681776002,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,...,1120.0,735.0,1088.0,991.5,931.0,1040.0,1040.0,1111.0,1080.0,1089.0
1,2,1681776003,1033.0,1051.0,1163.0,974.0,993.0,1095.0,1044.0,1152.0,...,1104.5,765.0,1090.0,984.0,928.5,1043.0,1052.0,1105.5,1075.0,1095.0
2,3,1681776004,1022.0,1051.0,1160.0,975.5,974.0,1081.0,1049.0,1133.0,...,1089.0,762.0,1095.0,993.0,926.0,1039.0,1033.0,1100.0,1021.0,1093.0
3,4,1681776005,1034.0,1050.0,1165.0,977.0,988.0,1084.0,1019.0,1128.0,...,1082.0,762.0,1100.0,999.0,929.0,1047.5,1014.0,1107.0,1059.5,1110.0
4,5,1681776006,1037.5,1049.0,1155.0,977.0,971.0,1084.0,1027.0,1144.0,...,1070.0,763.0,1100.0,1004.5,944.0,1056.0,1019.0,1098.0,1098.0,1107.5


In [18]:
df.shape


(83413, 26)

In [19]:
leak_locations_and_rateDF = pd.read_csv("Gasleak Data Sets/leak_locations_and_rate.csv")
leak_locations_and_rateDF.head()


,EventID,NumberSourcesLeaking,LeakPointId,Latitude,Longitude,EmissionCategory,UTCStart,UTCEnd,Duration,LeakRate,BFT,tStart,tEnd
0,40,2,4T-31,40.595830,-105.139824,Fugitive,2023-04-18_00:00:35.000000,2023-04-18_00:15:35.000000,900,36.636270,ReferenceMeteredEvent,1681776035,1681776935
1,40,2,5S-26,40.595924,-105.139390,Fugitive,2023-04-18_00:00:35.000000,2023-04-18_00:15:35.000000,900,64.621765,ReferenceMeteredEvent,1681776035,1681776935
2,41,2,4W-27,40.595936,-105.140270,Fugitive,2023-04-18_00:15:35.000000,2023-04-18_01:00:35.000000,2700,55.385376,ReferenceMeteredEvent,1681776935,1681779635
3,41,2,4S-43,40.595642,-105.140350,Fugitive,2023-04-18_00:15:35.000000,2023-04-18_01:00:35.000000,2700,24.345577,ReferenceMeteredEvent,1681776935,1681779635
4,42,2,4T-31,40.595830,-105.139824,Fugitive,2023-04-18_00:15:35.000000,2023-04-18_00:45:35.000000,1800,36.636270,ReferenceMeteredEvent,1681776935,1681778735


In [20]:
weather_df = pd.read_csv("Gasleak Data Sets/weather_data.csv")

In [21]:
total_data  = df.join(weather_df)
total_data = total_data.drop('timestamp', axis=1)
total_data.head()

,Unnamed: 0,time,111111_ 40.595561_-105.14055_3,111111_ 40.596108_-105.140583_4,111111_40.595556_-105.140069_2,111111_40.596114_-105.140075_1,222222_ 40.596108_-105.140583_4,222222_40.595556_-105.140069_2,222222_40.595561_-105.14055_3,222222_40.596114_-105.140075_1,...,555555_40.596097_-105.139678_4,666666_40.595542_-105.139211_2,666666_40.595547_-105.139714_3,666666_40.596089_-105.139144_1,666666_40.596097_-105.139678_4,Barometric_Pressure,Humidity,Temperature,Wind_Direction,Wind_Speed
0,1,1681776002,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,...,1040.0,1040.0,1111.0,1080.0,1089.0,827.064433,32.200298,10.971487,306.335774,1.836502
1,2,1681776003,1033.0,1051.0,1163.0,974.0,993.0,1095.0,1044.0,1152.0,...,1043.0,1052.0,1105.5,1075.0,1095.0,827.073897,32.200297,11.116626,311.523873,1.875021
2,3,1681776004,1022.0,1051.0,1160.0,975.5,974.0,1081.0,1049.0,1133.0,...,1039.0,1033.0,1100.0,1021.0,1093.0,827.301068,32.138772,10.861056,324.694315,1.966547
3,4,1681776005,1034.0,1050.0,1165.0,977.0,988.0,1084.0,1019.0,1128.0,...,1047.5,1014.0,1107.0,1059.5,1110.0,827.073913,32.100913,11.012511,316.473451,2.060548
4,5,1681776006,1037.5,1049.0,1155.0,977.0,971.0,1084.0,1027.0,1144.0,...,1056.0,1019.0,1098.0,1098.0,1107.5,827.092847,31.999946,10.969914,315.187093,1.947818


In [22]:
leak_locations_and_rateDF.head()

,EventID,NumberSourcesLeaking,LeakPointId,Latitude,Longitude,EmissionCategory,UTCStart,UTCEnd,Duration,LeakRate,BFT,tStart,tEnd
0,40,2,4T-31,40.595830,-105.139824,Fugitive,2023-04-18_00:00:35.000000,2023-04-18_00:15:35.000000,900,36.636270,ReferenceMeteredEvent,1681776035,1681776935
1,40,2,5S-26,40.595924,-105.139390,Fugitive,2023-04-18_00:00:35.000000,2023-04-18_00:15:35.000000,900,64.621765,ReferenceMeteredEvent,1681776035,1681776935
2,41,2,4W-27,40.595936,-105.140270,Fugitive,2023-04-18_00:15:35.000000,2023-04-18_01:00:35.000000,2700,55.385376,ReferenceMeteredEvent,1681776935,1681779635
3,41,2,4S-43,40.595642,-105.140350,Fugitive,2023-04-18_00:15:35.000000,2023-04-18_01:00:35.000000,2700,24.345577,ReferenceMeteredEvent,1681776935,1681779635
4,42,2,4T-31,40.595830,-105.139824,Fugitive,2023-04-18_00:15:35.000000,2023-04-18_00:45:35.000000,1800,36.636270,ReferenceMeteredEvent,1681776935,1681778735


In [23]:
leak_locations_and_rateDF = leak_locations_and_rateDF[["tStart", "tEnd"]]
leak_locations_and_rateDF.head()

,tStart,tEnd
0,1681776035,1681776935
1,1681776035,1681776935
2,1681776935,1681779635
3,1681776935,1681779635
4,1681776935,1681778735


In [24]:
leak_locations_and_rateDF.shape

(98, 2)

In [25]:
# Merge the DataFrames
merged_df = pd.merge(total_data, leak_locations_and_rateDF, how='cross')

# Convert Unix timestamps to datetime objects
merged_df['unix_time'] = pd.to_datetime(merged_df['time'], unit='s')
merged_df['start_time'] = pd.to_datetime(merged_df['tStart'])
merged_df['end_time'] = pd.to_datetime(merged_df['tEnd'])

# Create a new column with a binary value indicating if the time is in the range
merged_df['in_range'] = (merged_df['unix_time'] >= merged_df['start_time']) & (merged_df['unix_time'] <= merged_df['end_time'])
merged_df['in_range'] = merged_df['in_range'].astype(int)

# Drop unnecessary columns (if needed)
merged_df = merged_df.drop(['start_time', 'end_time'], axis=1)

In [26]:
merged_df.head()

,Unnamed: 0,time,111111_ 40.595561_-105.14055_3,111111_ 40.596108_-105.140583_4,111111_40.595556_-105.140069_2,111111_40.596114_-105.140075_1,222222_ 40.596108_-105.140583_4,222222_40.595556_-105.140069_2,222222_40.595561_-105.14055_3,222222_40.596114_-105.140075_1,...,666666_40.596097_-105.139678_4,Barometric_Pressure,Humidity,Temperature,Wind_Direction,Wind_Speed,tStart,tEnd,unix_time,in_range
0,1,1681776002,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,...,1089.0,827.064433,32.200298,10.971487,306.335774,1.836502,1681776035,1681776935,2023-04-18 00:00:02,0
1,1,1681776002,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,...,1089.0,827.064433,32.200298,10.971487,306.335774,1.836502,1681776035,1681776935,2023-04-18 00:00:02,0
2,1,1681776002,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,...,1089.0,827.064433,32.200298,10.971487,306.335774,1.836502,1681776935,1681779635,2023-04-18 00:00:02,0
3,1,1681776002,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,...,1089.0,827.064433,32.200298,10.971487,306.335774,1.836502,1681776935,1681779635,2023-04-18 00:00:02,0
4,1,1681776002,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,...,1089.0,827.064433,32.200298,10.971487,306.335774,1.836502,1681776935,1681778735,2023-04-18 00:00:02,0


In [27]:
merged_df.shape

(8174474, 35)